In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!wget https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2 -O rdkit.tar.bz2 -q
!mkdir rdkit
!tar xjf rdkit.tar.bz2 -C rdkit

!cp -r rdkit/lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!cp -r rdkit/lib/*.so.* /usr/lib/x86_64-linux-gnu/

!ln -s /usr/lib/x86_64-linux-gnu/libboost_python3-py36.so.1.65.1 /usr/lib/x86_64-linux-gnu/libboost_python3.so.1.65.1

mkdir: cannot create directory ‘rdkit’: File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libboost_python3.so.1.65.1': File exists


In [0]:
! pip install tensorflow==1.14
! pip install tensorflow-gpu==1.14

     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 491kB 49.6MB/s 
     |████████████████████████████████| 3.2MB 52.2MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 377.0MB 44kB/s 


In [0]:
import rdkit
from rdkit import Chem


In [0]:
import tensorflow as tf

In [0]:
%matplotlib inline
import pandas as pd
import keras
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.optimizers import Adam
#from keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import ReduceLROnPlateau
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint
from datetime import datetime

In [0]:
data = pd.read_csv('/content/drive/My Drive/Images_zinc/smiles_zinc.csv', skip_blank_lines=True, nrows=25000, usecols=[0])
from collections import deque
de = deque([])
charset = list("".join(list(data.smiles)))
for i in charset:
  if i not in de:
    de.append(i)
de.appendleft('E')

smile2index = dict((c,i) for i,c in enumerate(de))
index2smile= dict((i,c) for i,c in enumerate(de))

def smiles_encoder(smiles, maxlen=110):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
    X = np.zeros((maxlen, len(de)))
    for i, c in enumerate(smiles):
        X[i, smile2index[c]] = 1
    return X
 
def smiles_decoder(X):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smile[i]
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi.rstrip('E')))+'\n'

print(de)
data['smile_enc'] = data['smiles'].apply(lambda smiles: smiles_encoder(smiles))
print(data['smile_enc'][2000])
data['smile_check_dec'] = data['smile_enc'].apply(lambda smile_enc: smiles_decoder(smile_enc))
print(data['smiles'][1000])
print(data['smile_check_dec'][1000])


deque(['E', 'C', '(', ')', 'c', '1', '2', 'o', '=', 'O', 'N', '3', 'F', '\n', '[', '@', 'H', ']', 'n', '-', '#', 'S', 'l', '+', 's', 'B', 'r', '/', '4', '\\', '5', '6', '7', 'I', 'P'])
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[NH3+][C@@H](CSCc1nccs1)C(=O)[O-]

[NH3+][C@@H](CSCc1nccs1)C(=O)[O-]



In [0]:
data[:5]

,smiles,smile_enc,smile_check_dec
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


In [0]:
x_test = data['smile_enc'][2000:3000]
y_test = data['smile_enc'][2000:3000]
x_train = np.array(list(data['smile_enc'][:2000]))
y_train = np.array(list(data['smile_enc'][:2000]))
print(x_train[:,:,:][5])

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

AttributeError: ignored

In [0]:
batch_size = 8
image_size = x_train.shape[1]
original_dim = image_size
intermediate_dim = 3200
latent_dim = 15
epochs = 150

In [0]:
x =  Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim)) # for random_normal, mean=0. and std=1.
    return z_mean + K.exp(z_log_sigma) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# end-to-end autoencoder
vae = Model(x, x_decoded_mean)

# encoder, from inputs to latent space
encoder = Model(x, z_mean)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

def vae_loss(x, x_decoded_mean):
    xent_loss = metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (8, 110)             0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (8, 3200)            355200      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (8, 15)              48015       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (8, 15)              48015       dense_1[0][0]             

In [0]:
history = vae.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, x_test),
        )
print(history.history.keys())

ValueError: ignored